In [10]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from albumentations import HorizontalFlip, VerticalFlip,Rotate

In [11]:

def create_dir(path):
    """ Create a directory. """
    if not os.path.exists(path):
        os.makedirs(path)

In [12]:


def load_data(path, split=0.15):
    """ Load the images and masks """
    images = sorted(glob(f"{path}/images/*.png"))
    masks = sorted(glob(f"{path}/masks/*.png"))

    """ Split the data """
    split_size = int(len(images) * split)
    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=split_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y)

In [13]:



def augment_data(images, masks, save_path, augment=True):
    """ Performing data augmentation. """
    H = 128
    W = 128

    for idx, (x, y) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        """ Extracting the dir name and image name """
        #dir_name = x.split("\\")[-1]
        name = x.split("\\")[-1].split(".")[0]
      

        """ Read the image and mask """
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        y = cv2.imread(y, cv2.IMREAD_COLOR)

        if augment == True:
            aug = HorizontalFlip(p=1)
            augmented = aug(image=x, mask=y)
            x1 = augmented["image"]
            y1 = augmented["mask"]

            aug = VerticalFlip(p=1)
            augmented = aug(image=x, mask=y)
            x2 = augmented['image']
            y2 = augmented['mask']

            aug = Rotate(limit=45, p=1)
            augmented = aug(image=x, mask=y)
            x3 = augmented["image"]
            y3 = augmented["mask"]

            X = [x, x1, x2,x3]
            Y = [y, y1, y2,y3]

        else:
            X = [x]
            Y = [y]
        

        idx = 0
        for i, m in zip(X, Y):
            i = cv2.resize(i, (W, H))
            m = cv2.resize(m, (W, H))
            m = m/255.0
            m = (m > 0.5) * 255

            if len(X) == 1:
                tmp_image_name = f"{name}.jpg"
                tmp_mask_name  = f"{name}.jpg"
            else:
                tmp_image_name = f"{name}_{idx}.jpg"
                tmp_mask_name  = f"{name}_{idx}.jpg"

            image_path = os.path.join(save_path, "image/", tmp_image_name)
            mask_path  = os.path.join(save_path, "mask/", tmp_mask_name)
    

            cv2.imwrite(image_path, i)
            cv2.imwrite(mask_path, m)


            idx += 1


In [14]:
""" Load the dataset """
dataset_path = os.path.join("train")
dataset_path

'test'

In [15]:
(train_x, train_y), (valid_x, valid_y) = load_data(dataset_path, split=0.01)

In [16]:


    print("Train: ", len(train_x))
    print("Valid: ", len(valid_x))


Train:  456
Valid:  4


In [17]:

    create_dir("new_data/train/image/")
    create_dir("new_data/train/mask/")
    create_dir("new_data/valid/image/")
    create_dir("new_data/valid/mask/")


In [18]:
    augment_data(train_x, train_y, "new_data/train/", augment=False)
    augment_data(valid_x, valid_y, "new_data/valid/", augment=False)

100%|███████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 181.71it/s]
